In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

In [2]:
def zero_pad(X, pad):
    X_pad = np.pad(X, ((0, 0), (pad, pad), (pad, pad), (0, 0)), mode='constant', constant_values=(0, 0))
    return X_pad

In [3]:
def conv_single_step(a_slice_prev, W, b):
    
    s = a_slice_prev*W
    Z = np.sum(s)
    Z = Z + float(b)
    
    return Z

In [4]:
def conv_forward(A_prev, W, b, h_params):
    
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    (f, f, n_C_prev, n_C) = W.shape
    
    stride = h_params["stride"]
    pad = h_params["pad"]
    
    n_H = int((n_H_prev - f + 2*pad)/stride) + 1
    n_W = int((n_W_prev - f + 2*pad)/stride) + 1
    
    Z = np.zeros((m, n_H, n_W, n_C))
    
    A_prev_pad = zero_pad(A_prev, pad)
    
    for i in range(m):
        a_prev_pad = A_prev_pad[i, :, :, :]
        for h in range(n_H):
            vert_start = h*stride
            vert_end = h*stride + f
            
            for w in range(n_W):
                horiz_start = w*stride
                horiz_end = w*stride + f
                
                for c in range(n_C):
                    
                    a_slice_prev = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]
                    
                    weights = W[:, :, :, c]
                    biases = b[:, :, :, c]
                    Z[i, h, w, c] = conv_single_step(a_slice_prev, weights, biases)
    
    cache = (A_prev, W, b, h_params)
    
    return Z, cache

In [7]:
def pool_forward(A_prev, hparams, mode):
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    f = hparams['f']
    stride = hparams["stride"]
    
    n_H = int(1+(n_H_prev - f)/stride)
    n_W = int(1+(n_W_prev - f)/stride)
    n_C = n_C_prev
    
    A = np.zeros((m, n_H, n_W, n_C))
    
    for i in range(m):
        for h in range(n_H):
            vert_start = h*stride
            vert_end = h*stride + f
            
            for w in range(n_W):
                horiz_start = w*stride
                horiz_end = w*stride + f
                
                for c in range(n_C):
                    
                    a_prev_slice = A_prev[i, vert_start:vert_end, horiz_start:horiz_end, c]
                    
                    if mode == "max":
                        A[i, h, w, c] = np.max(a_prev_slice)
                    elif mode == "average":
                        A[i, w, h, c] = np.mean(a_prev_slice)
    
    cache = (A_prev, hparams)
    
    return A, cache

In [ ]:
def conv_backwards(dZ, cache):
    (A_prev, W, b, hparams) = cache
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    (f, f, n_C_prev, n_C) = W.shape
    
    stride = hparams["stride"]
    pad = hparams["pad"]
    
    (m, n_H, n_W, n_C) = dZ.shape
    
    dA_prev = np.zerps((m, n_H_prev, n_W_prev, n_C_prev))
    dW = np.zeros((f, f, n_C_prev, n_C))
    db = np.zeros((1, 1, 1, n_C))
    
    A_prev_pad = zero_pad(A_prev, pad)
    dA_prev_pad = zeropad(dA_prev, pad)
    
    for i in range(m):